# Rurality

> Attach different rurality indicators to InfoGroup establishments.



In [ ]:
#default_exp rurality

In [ ]:
#export
import json

import numpy as np
import pandas as pd
import fastparquet

from rurec import resources
from rurec.resources import Resource
from rurec import infogroup
from rurec import ers_codes

In [ ]:
# Register new resources created in this module
for y in range(1997, 2018):
    resources.add(Resource(f'infogroup/rural/{y}', f'/InfoGroup/data/rurality/{y}.pq', f'InfoGroup data with rural codes, {y}, parquet format', False))

## Open cleaned InfoGroup

In [ ]:
df_ig = infogroup.get_df(2010, cols=['YEAR', 'ABI', 'FIPS_CODE'])

## Merge in different geo codes, create new variables etc.

In [ ]:
df = ers_codes.get_ruc_df()
df = df.loc[df['RUC_YEAR'] == 2013, ['FIPS', 'RUC_CODE', 'RUC_YEAR']]
df.rename(columns={'FIPS': 'FIPS_CODE'}, inplace=True)
df_ig = df_ig.merge(df, 'left', 'FIPS_CODE')

## Save InfoGroup with new variables

In [ ]:
fastparquet.write(resources.get(f'infogroup/rural/2010').path, df_ig, write_index=False)

## Module's interface to get dataframe with rurality variables

In [ ]:
#export
def get_df(year, cols=None):
    """Return one year of InfoGroup with rurality variables.
    Subset of columns can be loaded by passing list to `cols`.
    """
    res = resources.get(f'infogroup/rural/{year}')
    return pd.read_parquet(res.path, 'fastparquet', columns=cols)